# Saildrone temperature logger processing code
- This code reads in the full saildrone dataset to get start/stop times and metadata
- It reads in the Saildrone provided .csv temperature logger files
- There are several dictionaries that contain filename, serial number, depth data
- These are then combined into a single file that matches the indices of the original saildrone data
- lat and lon from the saildrone data are added to the file

In [5]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seawater as sw
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features
import uuid
from scipy import interpolate
from glob import glob

#create xarray dataset with saildrone filenames
#data directory for saildrone data
data_dir = 'F:/data/cruise_data/saildrone/2020_atomic/'
saildrone_filename=[data_dir+'/delay_mode/1060/1min/saildrone-gen_5-atomic_eurec4a_2020-sd1060-20200117T000000-20200302T235959-1_minutes-v1.1589306886594.nc',
                    data_dir+'/delay_mode/1061/1min/saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1589307121602.nc',
                    data_dir+'/delay_mode/1026/1min/saildrone-gen_5-atomic_eurec4a_2020-sd1026-20200117T000000-20200302T235959-1_minutes-v1.1589306725934.nc'
                   ]
saildrone_filename_short=['saildrone-gen_5-atomic_eurec4a_2020-sd1060-20200117T000000-20200302T235959-1_minutes-v1.1589306886594.nc',
                          'saildrone-gen_5-atomic_eurec4a_2020-sd1061-20200117T000000-20200302T235959-1_minutes-v1.1589307121602.nc',
                          'saildrone-gen_5-atomic_eurec4a_2020-sd1026-20200117T000000-20200302T235959-1_minutes-v1.1589306725934.nc'
                         ]

usvs=[1060,1061,1026]

ds_info_saildrone = xr.Dataset(data_vars={'fname':(['trajectory'],saildrone_filename),'fname_short':(['trajectory'],saildrone_filename_short)},coords={'trajectory':usvs})


#data direcgtory for temperature logger .csv files
adir_sbe='F:/data/cruise_data/saildrone/2020_atomic/keel-mount-temp-loggers/'

#get list of all filenames in directory
files = [x for x in glob(adir_sbe+'*.csv')]
print('number of file:',len(files))
#print(files)

#time_start = '2019-05-14T23:00:00'
#time_end = '2019-10-11T18:30:00'


number of file: 11


In [6]:
#tem=df[df.vehicle==1060] 
#tem=tem[tem.serial_num==76102]
#df[df.serial_num==76102].depth.values
#print(tem.depth.values)
#iserial=76101
#df[df.serial_num==iserial].depth.values[0]

# make ds_info to store all the temperature logger info

In [7]:
#get list of all filenames in directory
#files = [x for x in glob(adir_sbe+'*.csv')]
#print('number of file:',len(files))
#RBR Solot
#Special Note: S/N 76106 unexpectedly stopped recording before the mission days began
data = {'vehicle':  [1060,1060,1060,1060,1061,1061,1061,1061,1026,1026,1026,1026],
        'serial_num': [76102,101317,76104,76106,101315,76111,100036,76110,76108,76109,100038,76103],
        'depth': [.4,.85,1.3,1.8,.4,.85,1.3,1.8,.4,.85,1.3,1.8]
        }
df = pd.DataFrame (data, columns = ['vehicle','serial_num','depth'])

#df[df.vehicle==1026].serial_num
adepth=[]
astr=[]
anum=[]

for iusv in usvs:
    adepth2=[]
    astr2=[]
    anum2=[]
    for file in files:
        ii=file.find('\\')
        iserial=int(file[ii+1:ii+7])
        #print(iserial)
        idepth=df[df.serial_num==iserial].depth.values[0]
        iusv2=df[df.serial_num==iserial].vehicle.values[0]
        if iusv2==iusv:
            astr2.append(file[ii+1:])
            anum2.append(iserial)
            adepth2.append(idepth)
    if iusv==1060:
        astr2.append('not_exist')
        anum2.append(76106)
        adepth2.append(1.8)
    sort_index = np.argsort(adepth2)
    #sort by depth
    adepth3,astr3,anum3,=[],[],[]
    for i in range(len(adepth2)):
        adepth3.append(adepth2[sort_index[i]])
        astr3.append(astr2[sort_index[i]])
        anum3.append(anum2[sort_index[i]])
    adepth.append(adepth3)
    astr.append(astr3)
    anum.append(anum3)

ds_info = xr.Dataset(data_vars={'fname':(['trajectory','z'],astr),
                                'depth':(['trajectory','z'],adepth),
                                'serial':(['trajectory','z'],anum)},
                     coords={'z':np.arange(4),'trajectory':usvs})


# Metadata
- SBE56 info, set some attributes and depths for metadata
- put filenames and depts into a xarray dataset


In [8]:
encoding_str={
      'sea_water_temperature_00': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_01': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_02': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_03': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_04': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_05': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_06': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'time':{'dtype':'float64', '_FillValue': -9999},
      'latitude':{'dtype':'float64', '_FillValue': -9999},
      'longitude':{'dtype':'float64', '_FillValue': -9999}
      }

#lat_attrs = {"_FillValue":np.nan,
lat_attrs = {'long_name':'latitude',
             'standard_name':'latitude',
             'units':'degrees_north',
             'coverage_content_type':'coordinate',
             'axis':'Y',
             'valid_max':90.0,
             'valid_min':-90.0}
             
#lon_attrs = {"_FillValue":np.nan,
lon_attrs = {'long_name':'longitude',
             'standard_name':'longitude',
             'units':'degrees_east',
             'coverage_content_type':'coordinate',
             'axis':'X',
             'valid_max':180.0,
             'valid_min':-180.0}

vattrs = {'long_name': 'sea surface temperature at',
          'standard_name': 'sea_water_temperature', 'installed_height':'-0.295 m' ,
          'serial_number' : 'RBR05608196', 'model_number': 'solo_t', 
          'sensor_description': 'RBR Solo Temperature Logger',
          'model_name': 'RBR_Solo_t', 'product_page':'https://rbr-global.com/products/compact-loggers/rbrsolo-t', 
          'nominal_sampling_schedule': '1 sec', 
          'units':'degrees_C','valid_min':-5.0,'valid_max':50.0}#,'_FillValue':-9.9}

tattrs = {'long_name':'time',
          'standard_name':'time',
          'coverage_content_type':'coordinate',
          'axis':'T'}

encoding_str_log={
      'sea_water_temperature_00_mean': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_01_mean': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_02_mean': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_03_mean': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_04_mean': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_05_mean': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_06_mean': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_00_std': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_01_std': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_02_std': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_03_std': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_04_std': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_05_std': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'sea_water_temperature_06_std': {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9999},
      'time':{'dtype':'float64', '_FillValue': -9999},
      'latitude':{'dtype':'float64', '_FillValue': -9999},
      'longitude':{'dtype':'float64', '_FillValue': -9999}
      }


# Combine the .cvs files into a single netcdf file with metadata

- for sbe3 on 1036, it fell off, so i put nan for that one
- for several 1037 they stopped at different dates so i filled with nan at end to make same length

In [ ]:
ds_sbe=[]
del ds_sbe
iobs_start=0
for iusv,usv in enumerate(usvs):   #loop over vehicles
    print(usv,iusv)
#    if iusv==0:
#        continue
    fname = str(ds_info_saildrone.sel(trajectory=usv).fname.data)
#    fname=saildrone_filename[iusv]
#    fname_out=sbe_filename[iusv]
    ds = xr.open_dataset(fname)
    ds.close()
    gattrs=ds.attrs 
#    time_start,time_stop = ds.time[0,0].data,ds.time[0,-1].data
    
    for iz in range(7):   #loop over SBEs 
        
        #vname='sea_water_temperature_'+str(iz).zfill(2)
        print(usv,iz) #,xf.time.min().data,xf.time.max().data)
        
        #read in the data and rename variable
        if ((usv==1060) and (iz==3)) or iz>3:  #the third sbe56 fell off & file doesn't exist
            vname2='sea_water_temperature_'+str(iz).zfill(2)
            xf=xf.rename({vname:vname2})
            xf[vname2]=xf[vname2]*np.nan
            vname='sea_water_temperature_'+str(iz).zfill(2)
        else:
            #create the .csv SBE filename to read in
            vname='sea_water_temperature_'+str(iz).zfill(2)
            tem = ds_info.sel(trajectory=usv).isel(z=iz)
            filename = adir_sbe + str(tem.fname.data)
            print(filename)
            df = pd.read_csv(filename,header=0,names=['time','temp'],skiprows=16)
            tt = df['time']
            tt=tt.replace(" ","T")
            t2=pd.to_datetime(tt)
            df['time']=t2
            df.index=df['time']
            xf=df.to_xarray()
            xf=xf.rename({'temp':vname})    
            vattrs['installed_height']=str(-1*ds_info.sel(trajectory=usv).isel(z=iz).depth.data)+' m'
            vattrs['serial_number']=ds_info.sel(trajectory=usv).isel(z=iz).serial.data
            vattrs['long_name']='sea surface temperature at '+str(ds_info.sel(trajectory=usv).isel(z=iz).depth.data)+' meters depth'
            xf.attrs=vattrs

        if iz==0:
            time_start2,time_stop2 = ds.time[0,0].data,ds.time[0,-1].data
            time_start,time_stop = xf.time[0].data,xf.time[-1].data
            time_start = max([time_start,time_start2])
            time_stop = min([time_stop,time_stop2])


        xf = xf.sel(time=slice(time_start,time_stop))
#        print('TIME')
#        print(xf.time[0].data,xf.time[-1].data)
#        print(len(xf.time))
   #    tem = ds_sbe.where(ds_sbe.time>=np.datetime64('2020-01-17T00:00:00'),drop=True)
#    tem = tem.where(tem.time<=np.datetime64('2020-03-02T23:59:00'),drop=True)

        
        if iz==0:
            nobs_start=xf.time.size
            
        iobs=np.arange(xf.time.size)
        nobs=xf.time.size
        
        if nobs_start>nobs:
            xf_save = xf_save.rename({vname_save:vname})
            xf = xr.concat([xf.sel(time=slice(xf.time[0],xf.time[-2])),xf_save.sel(time=slice(xf.time[-1],xf_save.time[-1]))*np.nan],dim='time')
        
        #fix gap in data in 076110 file
        if iz==3 and usv==1061:
            xf0=xf.isel(time=slice(0,2983269))
            xf1=xf.isel(time=slice(2983269,len(xf.time)))
            xf_insert = xf_save.isel(time=slice(2983269,2983269+17000))
            xf_insert['sea_water_temperature_03']=xf_insert.sea_water_temperature_03*np.nan
            xf_fill=xr.concat([xf0,xf_insert,xf1],dim='time')
            xf = xf_fill
            
        if iz==0:
            td=np.ones((1,xf.time.size))*np.nan
            td[0,:]=xf[vname] #vname
            ds_sbe=xr.Dataset(data_vars={vname:(['trajectory','obs'],td)},coords={'trajectory':[usv]})
            ds_sbe[vname].attrs=vattrs
        else:
            td=np.ones((1,xf.time.size))*np.nan
            ds_sbe2=xr.Dataset(data_vars={vname:(['trajectory','obs'],td)},coords={'trajectory':[usv]})
            ds_sbe2[vname][0,:]=xf[vname].data
            ds_sbe2[vname].attrs=vattrs
            ds_sbe = xr.merge([ds_sbe,ds_sbe2])

        xf_save=xf
        vname_save=vname
        #print(ds_sbe.trajectory)
    td=np.zeros((1,xf.time.size), dtype='datetime64[s]')
    td[0,:]=xf.time.data
    tem=xr.Dataset(data_vars={'time':(['trajectory','obs'],td)},coords={'trajectory':[usv]})
    ds_sbe = xr.merge([ds_sbe,tem])
    ds_sbe=ds_sbe.assign_coords(time=ds_sbe.time)
    gattrs['time_coverage_resolution']='PT2S'
    gattrs['uuid']=str(uuid.uuid4())
    gattrs['keywords']='Temperature, Saildrone, Arctic, Bering Sea, Chukchi Sea, NASA, NOAA'
    gattrs['history']='created post-cruise by Gentemann 3/25/2020'  
    ds_sbe['trajectory']=ds.trajectory
    
#add lat and lon and time
    flat = interpolate.interp1d(ds.time[0,:].astype('float64'),ds.latitude[0,:])
    newlat = flat(ds_sbe.time[0,:].astype('float64'))
    flon = interpolate.interp1d(ds.time[0,:].astype('float64'),ds.longitude[0,:])
    newlon = flon(ds_sbe.time[0,:].astype('float64'))
    td=np.zeros((1,xf.time.size), dtype='double')
    td[0,:]=newlat
#    tem=xr.Dataset(data_vars={'latitude':(['trajectory','obs'],td)},coords={'trajectory':[usv]})
    tem=xr.Dataset(data_vars={'latitude':(['trajectory','obs'],td)},coords={'trajectory':ds_sbe.trajectory})
#    print('****')
#    print(tem)
#    print('****')
    ds_sbe = xr.merge([ds_sbe,tem])
    ds_sbe=ds_sbe.assign_coords(latitude=ds_sbe.latitude)
    td=np.zeros((1,xf.time.size), dtype='double')
    td[0,:]=newlon
    tem=xr.Dataset(data_vars={'longitude':(['trajectory','obs'],td)},coords={'trajectory':ds_sbe.trajectory})
    ds_sbe = xr.merge([ds_sbe,tem])
    ds_sbe=ds_sbe.assign_coords(longitude=ds_sbe.longitude)
    ds_sbe.latitude.attrs=lat_attrs
    ds_sbe.longitude.attrs=lon_attrs
    ds_sbe.time.attrs=tattrs
    ds_sbe.trajectory.attrs={'cf_role':'trajectory_id'}
    ds_sbe.attrs=gattrs

    
    fname = str(ds_info_saildrone.sel(trajectory=usv).fname_short.data)
    beg_fname = 'temperature_loggers-atomic_eurec4a_2020-sd'+str(usv)+'-20200117T000000-20200302T235959-1_second-v1.nc'
    fout=data_dir+'/temp_log_proc/'+beg_fname

#output data
    ds_sbe.to_netcdf(fout,encoding=encoding_str)
    del ds_sbe


# Create files with 1 minute averages of data using :54 to :06 seconds 

In [16]:
#make 1min average data
#iusv=0
#usv=1060
for iusv,usv in enumerate(usvs):   #loop over vehicles
#    if iusv==0:
#        continue
    print(usv,iusv)
    avehicle=str(usv)
    beg_fname = 'temperature_loggers-atomic_eurec4a_2020-sd'+str(usv)+'-20200117T000000-20200302T235959-1_second-v1.nc'
    filename=data_dir+'/temp_log_proc/'+beg_fname
    beg_fname = 'temperature_loggers-atomic_eurec4a_2020-sd'+str(usv)+'-20200117T000000-20200302T235959-1_minute-v1.nc'
    filename_out=data_dir+'/temp_log_proc/'+beg_fname
    xf=xr.open_dataset(filename)
    offset = pd.to_timedelta(6, unit='s') 
    xftem=xf
    xftem = xftem.isel(trajectory=0)
    xftem = xftem.swap_dims({'obs':'time'})
    #offset time by 6 seconds then mask all data outside first 12 which acutally is :54 to :06
    pt = pd.to_datetime(xftem.time.data+offset)
    psec=pt.second
    tem=xftem.where(psec <=12)  #data average should be only use sec 54 to sec 06 of each minute 
    tem['time']=tem['time']+offset #resample goes from :00 to :59 so shift +6sec
    #resample data to 1min
    xf_avg1 = tem.resample(time='1min').mean(keep_attrs=True)
    xf_std1 = tem.resample(time='1min').std(ddof=1,keep_attrs=True)
    for var in xf_avg1:
        var2=var+'_mean'
        sattrs=xf_avg1[var].attrs
        xf_avg1 = xf_avg1.rename({var:var2})
        sattrs['nominal_sampling_schedule']='1 minute averages from :54 to :06 seconds'
        sattrs['long_name']='sea surface depth 1-min mean'
        xf_avg1[var2].attrs=sattrs
        sattrs['long_name']='sea surface depth 1-min standard dev'
    for var in xf_std1:
        var2 = var+'_std'
        sattrs=xf_std1[var].attrs
        xf_std1 = xf_std1.rename({var:var2})
        sattrs['nominal_sampling_schedule']='1 minute averages from :54 to :06 seconds'
        sattrs['long_name']='sea surface depth 1-min standard dev'
        xf_std1[var2].attrs=sattrs
    
    ds_sbe=[]
    ds_usv = xr.open_dataset(saildrone_filename[0])
    ds_usv.close()
    gattrs=ds_usv.attrs 
    for ivname,vname in enumerate(xf_avg1):
        if ivname==0:
            td=np.ones((1,xf_avg1.time.size))*np.nan
            td[0,:]=xf_avg1[vname].data #vname
            ds_sbe=xr.Dataset(data_vars={vname:(['trajectory','obs'],td)},coords={'trajectory':ds_usv.trajectory})
            ds_sbe[vname].attrs=xf_avg1[vname].attrs
        else:
            td=np.ones((1,xf_avg1.time.size))*np.nan
            td[0,:]=xf_avg1[vname].data #vname
            ds_sbe2=xr.Dataset(data_vars={vname:(['trajectory','obs'],td)},coords={'trajectory':ds_usv.trajectory})
            ds_sbe2[vname].attrs=xf_avg1[vname].attrs
            ds_sbe = xr.merge([ds_sbe,ds_sbe2])
    for ivname,vname in enumerate(xf_std1):
        td=np.ones((1,xf_std1.time.size))*np.nan
        td[0,:]=xf_std1[vname].data #vname
        ds_sbe2=xr.Dataset(data_vars={vname:(['trajectory','obs'],td)},coords={'trajectory':ds_usv.trajectory})
        ds_sbe2[vname].attrs=xf_std1[vname].attrs
        ds_sbe = xr.merge([ds_sbe,ds_sbe2])
        
    #add lat/lon from USV
    td=np.zeros((1,ds_usv.time.size), dtype='double')
    td[0,:]=ds_usv.latitude.data
    tem=xr.Dataset(data_vars={'latitude':(['trajectory','obs'],td)},coords={'trajectory':ds_sbe.trajectory})
    ds_sbe = xr.merge([ds_sbe,tem])
    td=np.zeros((1,ds_usv.time.size), dtype='double')
    td[0,:]=ds_usv.longitude.data
    tem=xr.Dataset(data_vars={'longitude':(['trajectory','obs'],td)},coords={'trajectory':ds_sbe.trajectory})
    ds_sbe = xr.merge([ds_sbe,tem])
    td=np.zeros((1,ds_usv.time.size), dtype='datetime64[ns]')
    td[0,:]=ds_usv.time.data
    tem=xr.Dataset(data_vars={'time':(['trajectory','obs'],td)},coords={'trajectory':ds_sbe.trajectory})
    ds_sbe = xr.merge([ds_sbe,tem])
    
    ds_sbe.latitude.attrs=lat_attrs
    ds_sbe.longitude.attrs=lon_attrs
    ds_sbe.time.attrs=tattrs
    ds_sbe.trajectory.attrs={'cf_role':'trajectory_id'}
    ds_sbe.attrs=gattrs

    ds_sbe.to_netcdf(filename_out,encoding=encoding_str_log)  


1061 1


C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\core\nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


1026 2


C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\core\nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
